# Find top 10 most similar results using USE.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import joblib
import numpy as np
import re
import time
from sklearn.metrics.pairwise import cosine_similarity
import os
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/StackOverflowData/pretrain_data_all/featurized_data.csv')
df.shape

(812363, 11)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/StackOverflowData/pretrain_data_all/featurized_data_V2.csv')
df.shape

(812363, 11)

In [ ]:
df.head()

,id,preprocessed_title,preprocessed_question,preprocessed_answer,score,tags,answer,question_content,all_text,question_length,title
0,123,java lib app convert csv xml file,existing application library java allow conver...,understand would want sounds almost like cargo...,17|5|19|31|20|17|9|14|9|4|16|5|71|9|4|49,java|xml|csv|data-conversion,<p>I don't understand why you would want to do...,java lib app convert csv xml file existing app...,java lib app convert csv xml file existing app...,25,Java lib or app to convert CSV to XML file?
1,845,detect one defined font used web page,suppose following css rule page could detect o...,using fount drag fount button bookmarks bar cl...,3|10|3|10|5|8|11|37|13|82|2,javascript|html|css|fonts,<p>I am using Fount. You just have to drag the...,detect one defined font used web page suppose ...,detect one defined font used web page suppose ...,55,How to detect which one of the defined font wa...
2,3404,javascript troubleshooting tools internet expl...,use firebug mozilla js console heavily every r...,use microsoft script debugger firebug lite dep...,3|2|5|4|1|30|20,javascript|internet-explorer|debugging,"<p>I use both <a href=""http://www.microsoft.co...",javascript troubleshooting tools internet expl...,javascript troubleshooting tools internet expl...,50,JavaScript Troubleshooting Tools in Internet E...
3,3713,call asp net function javascript,writing web page asp net javascript code submi...,try could run asp net method using jquery page...,0|0|0|0|2|4|2|0|1|0|3|98|18|61|0|11|1|5|0|1,javascript|c#|asp.net|onclick,<p>I try this and so I could run an Asp.Net me...,call asp net function javascript writing web p...,call asp net function javascript writing web p...,24,Call ASP.NET function from JavaScript?
4,4923,wrapping lists columns,using populate template includes html unordere...,flexbox used wrap items row column directions ...,4|4|14|6|5|2|3|3|30|5|2|18|5,javascript|jquery|html|css|cfml,<p>Flexbox can be used to wrap items in both r...,wrapping lists columns using populate template...,wrapping lists columns using populate template...,33,Wrapping lists into columns


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test = train_test_split(df,test_size=0.5,random_state=42)

# reset index of train and test
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

print('Shape of X_train',X_train.shape)
print('Shape of X_test',X_test.shape)

Shape of X_train (406181, 11)
Shape of X_test (406182, 11)


In [ ]:
X_train.head()

,id,preprocessed_title,preprocessed_question,preprocessed_answer,score,tags,answer,question_content,all_text,question_length,title
0,10511845,loaded simultaneously parallel,two simple js scripts loaded order js referenc...,downloads may may parallel depending browser p...,4,javascript|apache,<p>Downloads may or may not be in parallel (de...,loaded simultaneously parallel two simple js s...,loaded simultaneously parallel two simple js s...,40,Loaded Simultaneously in Parallel?
1,34365869,mean javascript,following lines code mean also address jshint ...,equivalent else statement true false replaced ...,0,javascript|jquery,<p>It's equivalent to an if{}else{} statement....,mean javascript following lines code mean also...,mean javascript following lines code mean also...,17,What Does ? : Mean in JavaScript
2,72294188,want retrive data tabular form using ajax php,written following code showing want show data ...,lies wrong probably file try execute provided ...,1,javascript|php|html|ajax|forms,<p>Here lies the wrong</p>\n<pre><code>/usr/bi...,want retrive data tabular form using ajax php ...,want retrive data tabular form using ajax php ...,29,I want to retrive data in tabular form using a...
3,46873095,add duplicate values map,know map allow duplicate values way add duplic...,add duplicate key values create add values col...,0|0|-1,java|collections,<p>You do not add duplicate key values to a <c...,add duplicate values map know map allow duplic...,add duplicate values map know map allow duplic...,36,How to add duplicate Values to a map?
4,37178909,java swing jmenu icon shifts right,add icon jmenu applet run applet icon appear a...,adding solved problem least though menu text s...,0,java|swing|jmenu,<p>Adding this solved my problem at least:</p>...,java swing jmenu icon shifts right add icon jm...,java swing jmenu icon shifts right add icon jm...,28,Java Swing JMenu's icon shifts to right


In [ ]:
X_train_drop = X_train

In [ ]:
X_train_drop.head()

,id,preprocessed_title,preprocessed_question,preprocessed_answer,score,tags,answer,question_content,all_text,question_length,title
0,10511845,loaded simultaneously parallel,two simple js scripts loaded order js referenc...,downloads may may parallel depending browser p...,4,javascript|apache,<p>Downloads may or may not be in parallel (de...,loaded simultaneously parallel two simple js s...,loaded simultaneously parallel two simple js s...,40,Loaded Simultaneously in Parallel?
1,34365869,mean javascript,following lines code mean also address jshint ...,equivalent else statement true false replaced ...,0,javascript|jquery,<p>It's equivalent to an if{}else{} statement....,mean javascript following lines code mean also...,mean javascript following lines code mean also...,17,What Does ? : Mean in JavaScript
2,72294188,want retrive data tabular form using ajax php,written following code showing want show data ...,lies wrong probably file try execute provided ...,1,javascript|php|html|ajax|forms,<p>Here lies the wrong</p>\n<pre><code>/usr/bi...,want retrive data tabular form using ajax php ...,want retrive data tabular form using ajax php ...,29,I want to retrive data in tabular form using a...
3,46873095,add duplicate values map,know map allow duplicate values way add duplic...,add duplicate key values create add values col...,0|0|-1,java|collections,<p>You do not add duplicate key values to a <c...,add duplicate values map know map allow duplic...,add duplicate values map know map allow duplic...,36,How to add duplicate Values to a map?
4,37178909,java swing jmenu icon shifts right,add icon jmenu applet run applet icon appear a...,adding solved problem least though menu text s...,0,java|swing|jmenu,<p>Adding this solved my problem at least:</p>...,java swing jmenu icon shifts right add icon jm...,java swing jmenu icon shifts right add icon jm...,28,Java Swing JMenu's icon shifts to right


In [ ]:
X_train_drop = X_train_drop.drop(columns=["preprocessed_title","preprocessed_question","question_content","all_text"])

In [ ]:
joblib.dump(X_train,'/content/drive/MyDrive/StackOverflowData/pretrain_data/X_train.csv')

['/content/drive/MyDrive/StackOverflowData/pretrain_data/X_train.csv']

In [ ]:
joblib.dump(X_train_drop,'/content/drive/MyDrive/StackOverflowData/pretrain_data_all/X_train_final.pkl')

['/content/drive/MyDrive/StackOverflowData/pretrain_data_all/X_train_final.pkl']

In [ ]:
joblib.dump(X_train_drop,'/content/drive/MyDrive/StackOverflowData/pretrain_data_all/X_train_final_V2.pkl')

['/content/drive/MyDrive/StackOverflowData/pretrain_data_all/X_train_final_V2.pkl']

In [ ]:
test = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data_all/X_train.pkl')

In [ ]:
test.shape

(406181, 11)

In [ ]:
# below mentioned functions are helper functions which is use for text preprocessing
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"\n", "", phrase)
    return phrase

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def remove_stopwords(text):
    '''this function will remove stopwords from text '''
    final_text = ''
    for word in text.split():
        if word not in stopwords:
            final_text += word + ' '
    return final_text

def preprocess_title(text):
    # convert to lower case
    text = text.lower()
    # decontract
    text = decontracted(text)
    # remove all punctuations except a-z and c# and c++
    text = re.sub('[^a-zc#c++]+',' ',text)
    # remove stop words
    text = remove_stopwords(text)
    return text

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
# load universal sentence encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
def embed_text(sentences):
    ''' This function will convert sentence into vector using universal sentence encoder'''
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        # create vectors from sentences
        embeddings = session.run(embed(sentences))
        # return  embeddings
        return embeddings

In [ ]:
# load X_train
X_train = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data/X_train.csv')
# load question embedding trained using universal sentence encoder
question_embeddings = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data/universal_embed/universal_question_embedding.pkl')

In [ ]:
# load X_train
X_train = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data/X_train.pkl')
# load question embedding trained using universal sentence encoder
question_embeddings = joblib.load('/content/drive/MyDrive/StackOverflowData/pretrain_data/universal_embed/universal_question_embedding.pkl')

In [ ]:
def final_func1(question,top_n=10):
    ''' This function will find top similar result for given query using universal sentence encoder'''
    start = time.time()
    # preprocess question
    question = preprocess_title(question)
    # initialize  vector for user query
    query_vector = embed_text([question])
    similarities =  cosine_similarity((query_vector).reshape(1, -1), Y=question_embeddings, dense_output=True)
    # sort similarities
    sort = np.argsort(similarities[0])
    # get top similarity indices  in descending order
    similarity_index = np.array(list(reversed(sort)))
    # find top n similarities
    top_similarity_index = similarity_index[:top_n]
    # print top similarity values
    print('Top cosine similarities are ======>',similarities[0][top_similarity_index])
    # get similar questions title
    similar_questions = X_train['title'][top_similarity_index]
    total_time = (time.time() - start)
    print('\t')
    print('Total time ===========> ',total_time)
    print('Top 10 similar questions Using Universal sentence encoder')
    print('='*100)
    return list(similar_questions)

In [ ]:
query = 'How to make dictionary in python'
final_func1(query)

Top cosine similarities are ======> [0.63726175 0.6081903  0.60663766 0.5889329  0.5588284  0.54123473
 0.53672063 0.5295532  0.50545824 0.5034981 ]
	
Total time ===========>  12.071171045303345
Top 10 similar questions Using Universal sentence encoder


['What is the Python 3 equivalent of "python -m SimpleHTTPServer"',
 'How can I remove a key from a Python dictionary?',
 'What does __all__ mean in Python?',
 'Create a dictionary with comprehension',
 'Replacements for switch statement in Python?',
 'How do I sort a list of dictionaries by a value of the dictionary?',
 "Does Python have a string 'contains' substring method?",
 'How do I merge two dictionaries in a single expression?',
 'How do I access environment variables in Python?',
 'How do I write JSON data to a file?']